In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from arch.unitroot import ADF, PhillipsPerron, KPSS

In [7]:
df = pd.read_excel("/Users/neseuygun/Desktop/data/ons.xlsx")
df

,Tarih,TP MK LON YTL
0,2000-06,284.90
1,2000-07,281.10
2,2000-08,274.60
3,2000-09,273.80
4,2000-10,270.40
...,...,...
273,2023-03,1927.60
274,2023-04,1994.37
275,2023-05,1982.26
276,2023-06,1953.50


In [9]:
df.rename(columns={"TP MK LON YTL":"Ons"}, inplace=True)
df.head()

,Tarih,Ons
0,2000-06,284.9
1,2000-07,281.1
2,2000-08,274.6
3,2000-09,273.8
4,2000-10,270.4


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Tarih   278 non-null    object 
 1   Ons     278 non-null    float64
dtypes: float64(1), object(1)
memory usage: 4.5+ KB


In [12]:
Y = df["Ons"]
Y

0       284.90
1       281.10
2       274.60
3       273.80
4       270.40
        ...   
273    1927.60
274    1994.37
275    1982.26
276    1953.50
277    1947.71
Name: Ons, Length: 278, dtype: float64

In [42]:
def unit_root_test(Yt):
    trend_model = ["n", "c", "ct"]
    # KPSS model has not include n trend model.
    column_names = ["Test Name", "Model Type", "Lags", "Test Statistics", "P_Value"]
    df = pd.DataFrame(columns=column_names)
    for i in trend_model:
        try:
            adf = ADF(Yt,trend=i, method="aic")
            pp = PhillipsPerron(Yt,trend=i)
            kpss = KPSS(Yt,trend=i)
            df = pd.concat([df,
                           pd.DataFrame([[adf._test_name, adf.trend, adf.lags, adf.stat, adf.pvalue]], columns=df.columns),
                           pd.DataFrame([[pp._test_name, pp.trend, pp.lags, pp.stat, pp.pvalue]], columns=df.columns),
                           pd.DataFrame([[kpss._test_name, kpss.trend, kpss.lags, kpss.stat, kpss.pvalue]], columns=df.columns)
                            ])
        except ValueError:
            df = pd.concat([df,
                           pd.DataFrame([[adf._test_name, adf.trend, adf.lags, adf.stat, adf.pvalue]], columns=df.columns),
                           pd.DataFrame([[pp._test_name, pp.trend, pp.lags, pp.stat, pp.pvalue]], columns=df.columns)
                            ])
    df = df.sort_values(by="Test Name").reset_index(drop=True)
    return df

      


In [43]:
unit_root_test(Y)

,Test Name,Model Type,Lags,Test Statistics,P_Value
0,Augmented Dickey-Fuller,n,1,1.385419,0.958135
1,Augmented Dickey-Fuller,c,1,-0.723516,0.840564
2,Augmented Dickey-Fuller,ct,1,-1.952966,0.626802
3,KPSS Stationarity Test,c,10,2.200796,0.000100
4,KPSS Stationarity Test,ct,10,0.268209,0.003282
5,Phillips-Perron Test,n,16,1.479112,0.965421
6,Phillips-Perron Test,c,16,-0.655332,0.857983
7,Phillips-Perron Test,ct,16,-1.934275,0.636728


With unit root tests we can check if a timeseries is a stationary or not. If P_Value is bigger than 0.05 for Augmented Dickey-Fuller and Phillips-Perron Test we can say the series has unit root and it is not stationary. For KPPS Stationary Test it is opposite so if P_Value is smaller than 0.05 we can say the series has unit root.